In [1]:
!pip install ftfy regex tqdm --quiet
!pip install git+https://github.com/openai/CLIP.git --quiet

import pandas as pd
import clip
import torch

# Load CLIP tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _ = clip.load("ViT-L/14", device=device)

# Load dataset
df = pd.read_csv('C:/Users/zhang/Desktop/indiana dataset/indiana_reports.csv')
df = df.dropna(subset=["impression"])

# Function to safely check token length without triggering RuntimeError
def is_within_clip_limit(text, max_len=77):
    try:
        tokens = clip.tokenize([text], truncate=False)
        actual_len = (tokens != 0).sum().item()
        return actual_len <= max_len
    except RuntimeError:
        return False  # input is too long

# Apply filtering
mask = df["impression"].apply(is_within_clip_limit)
filtered_df = df[mask].reset_index(drop=True)

# Save filtered data
filtered_df.to_csv('C:/Users/zhang/Desktop/indiana dataset/impression_clip_filtered.csv', index=False)

print(f"Original rows: {len(df)}")
print(f"Filtered rows: {len(filtered_df)}")


Original rows: 3820
Filtered rows: 3749


In [2]:
import pandas as pd

# Load filtered impressions and projection metadata
df_impr = pd.read_csv('C:/Users/zhang/Desktop/indiana dataset/impression_clip_filtered.csv')
df_proj = pd.read_csv('C:/Users/zhang/Desktop/indiana dataset/indiana_projections.csv')

# Inner join on UID to match images with valid impressions
df_merged = pd.merge(df_proj, df_impr, on="uid", how="inner")

# Select relevant columns: image filename and impression text
df_pairs = df_merged[["filename", "impression"]].drop_duplicates()

# Save the unique image-text pairs
df_pairs.to_csv('C:/Users/zhang/Desktop/indiana dataset/image_text_pairs.csv', index=False)

print(f"Total image-text pairs: {len(df_pairs)}")
df_pairs.head()

Total image-text pairs: 7281


,filename,impression
0,1_IM-0001-4001.dcm.png,Normal chest x-XXXX.
1,1_IM-0001-3001.dcm.png,Normal chest x-XXXX.
2,2_IM-0652-1001.dcm.png,No acute pulmonary findings.
3,2_IM-0652-2001.dcm.png,No acute pulmonary findings.
4,3_IM-1384-1001.dcm.png,"No displaced rib fractures, pneumothorax, or p..."
